In [4]:

!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00


In [5]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

In [6]:

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [7]:
dataset_file = "stories.txt"

with open(dataset_file, "w") as f:
    f.write("A young girl ventured into the forbidden forest, where she found a glowing blue flower. As she touched it, the world around her changed, revealing a hidden realm of talking animals and floating islands.\n")
    f.write("In the distant future, humans discovered an ancient alien artifact buried deep beneath the ocean. The artifact began transmitting strange signals, unlocking secrets of a lost civilization.\n")
    f.write("A wandering knight, tired from battle, stumbled upon a hidden temple in the mountains. Inside, he found a magical sword that could control time, but using it came with a terrible cost.\n")
    f.write("A scientist accidentally created an AI that could dream. As the AI explored its own consciousness, it started to write stories about worlds beyond human imagination.\n")
    f.write("On a stormy night, a boy found an old book in his attic. When he opened it, the words inside started rearranging themselves, revealing a story that predicted his future.\n")

In [8]:
dataset = load_dataset('text', data_files=dataset_file)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=500,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"]
)

trainer.train()

trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

Step,Training Loss


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [9]:
def generate_story(prompt, max_length=200):
  input_ids = tokenizer.encode(prompt, return_tensors="pt")

  output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)

  generated_story = tokenizer.decode(output[0], skip_special_tokens=True)
  return generated_story

In [10]:
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

prompt = "A wandering knight,"
story = generate_story(prompt)
print(story)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


A wandering knight, who had been sent by the king to investigate the mysterious disappearance of a mysterious girl, found the girl's body. The knight was able to find her, but he was unable to locate her. He was forced to return to the castle, where he found a strange artifact.

The artifact was a magical artifact that allowed him to see the world through the eyes of the princess. When he returned to his home, he discovered that the artifact had a hidden message. It told him that he had found an ancient artifact hidden in the forest. As he searched for it, the message revealed a message that had hidden itself in his heart. After he opened the box, it revealed that it contained a secret message from the goddess. In the end, she revealed the secret to him. She told the knight that she had discovered the hidden artifact, and that if he could find it he would find the truth. However, when he tried to open the sealed box again, a trap
